In [1]:
import cv2
import torch
import glob as gb
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import scipy.signal as signal
import scipy.fftpack as fftpack

In [2]:
# set path
datapaths = "../SMIC/SMIC_all_cropped/HS"   # define UCF-101 RGB data path
sample = "/s1/micro/negative/s1_ne_01/*"
Logpath = "Log"
save_path = "../OutPut"
action_names = ['negative', 'positive','surprise','non_micro-expression']

In [3]:
#Build Gaussian Pyramid
def build_gaussian_pyramid(src,level=3):
    s=src.copy()
    pyramid=[s]#pyramid size:1
#     print("pyramid_before",len(pyramid))
    for i in range(level):
#         print(i,"s_before",s.shape)
        s=cv2.pyrDown(s) #shrink
#         print(i,"s_after",s.shape)
        pyramid.append(s)
#     print("pyramid_after",len(pyramid))
    return pyramid #pyramid size:4

In [4]:
#Build Laplacian Pyramid
def build_laplacian_pyramid(src,levels=3):
    gaussianPyramid = build_gaussian_pyramid(src, levels)
    pyramid=[]
    for i in range(levels,0,-1):
#         print("i:", gaussianPyramid[i].shape,"i-1:", gaussianPyramid[i-1].shape)
        GE=cv2.pyrUp(gaussianPyramid[i])
        GD = gaussianPyramid[i-1]
#         print("i:", GE.shape,"i-1:", gaussianPyramid[i-1].shape)
#         GD=cv2.resize(gaussianPyramid[i-1],(GE.shape[1], GE.shape[0]), interpolation=cv2.INTER_CUBIC)
        GE=cv2.resize(GE,(GD.shape[1], GD.shape[0]), interpolation=cv2.INTER_CUBIC)
#         print("GE:", GE.shape, "GD", GD.shape)
        L=cv2.subtract(GD,GE)
#         print("substract:", L.shape)
        pyramid.append(L)
    return pyramid

In [5]:
#load video from file
def load_video(video_filename):
    
    cap = gb.glob(video_filename)#open image sequence file    
    frame_count = len(cap)
    print("frame_count",frame_count)
    width, height,channel = cv2.imread(cap[0]).shape
    video_tensor = np.zeros((frame_count,width, height,channel),dtype='float')
    x=0
    for img in cap:
        frame = cv2.imread(img)
        video_tensor[x]=frame
        x+=1
    fps = 100
    print("videotensor:",video_tensor.shape)
    return video_tensor,fps

In [6]:
#build laplacian pyramid for video
def laplacian_video(video_tensor,levels=3):
    tensor_list=[]
    for i in range(0,video_tensor.shape[0]):
        frame=video_tensor[i]
        pyr=build_laplacian_pyramid(frame,levels=levels)
#         print(i,"pyr:", len(pyr), "py0:", pyr[0].shape,"py1:", pyr[1].shape,"py2:", pyr[2].shape)
        if i==0:
            for k in range(levels):
                tensor_list.append(np.zeros((video_tensor.shape[0],pyr[k].shape[0],pyr[k].shape[1],3)))
#                 print(k,tensor_list[k].shape)
        for n in range(levels):
            tensor_list[n][i] = pyr[n]
    return tensor_list

In [7]:
#butterworth bandpass filter
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    omega = 0.5 * fs
    low = lowcut / omega
    high = highcut / omega
    b, a = signal.butter(order, [low, high], btype='band')
    y = signal.lfilter(b, a, data, axis=0)
    return y

In [8]:
def reconstract_from_tensorlist(filter_tensor_list,levels=3):
    final=np.zeros(filter_tensor_list[-1].shape)
    for i in range(filter_tensor_list[0].shape[0]):
        up = filter_tensor_list[0][i]
        for n in range(levels-1):
            up=cv2.pyrUp(up)
            uf = filter_tensor_list[n + 1][i]
            up = cv2.resize(up,(uf.shape[1], uf.shape[0]), interpolation=cv2.INTER_CUBIC)
#             up=cv2.pyrUp(up)+#可以改为up=cv2.pyrUp(up)
            up+=uf
#             print("up:", up.shape,"ftl:", filter_tensor_list[n + 1][i].shape)
        final[i]=up
    return final

In [9]:
def save_image(final):   
    save_name = "MagImg"
    ext = ".png"
    for i in range(final.shape[0]):
        sample = final[i]
        filename =save_name+ str(i)+ext
        print(filename)
        cv2.imwrite(filename, sample)

In [10]:
#manify motion
def magnify_motion(datapaths,low,high,levels=3,amplification=20):
    video_name = datapaths+sample
    t,f=load_video(video_name)    
    lap_video_list=laplacian_video(t,levels=levels)

    filter_tensor_list=[]
    for i in range(levels):
        filter_tensor=butter_bandpass_filter(lap_video_list[i],low,high,f)
        filter_tensor*=amplification
        filter_tensor_list.append(filter_tensor)
    recon=reconstract_from_tensorlist(filter_tensor_list)    
    final=t+recon
    print('final:',final.shape)
    save_image(final)
#     save_video(final)


In [11]:
if __name__=="__main__":
    magnify_motion(datapaths,0.15,0.3)


frame_count 36
videotensor: (36, 161, 131, 3)
final: (36, 161, 131, 3)
MagImg0.png
MagImg1.png
MagImg2.png
MagImg3.png
MagImg4.png
MagImg5.png
MagImg6.png
MagImg7.png
MagImg8.png
MagImg9.png
MagImg10.png
MagImg11.png
MagImg12.png
MagImg13.png
MagImg14.png
MagImg15.png
MagImg16.png
MagImg17.png
MagImg18.png
MagImg19.png
MagImg20.png
MagImg21.png
MagImg22.png
MagImg23.png
MagImg24.png
MagImg25.png
MagImg26.png
MagImg27.png
MagImg28.png
MagImg29.png
MagImg30.png
MagImg31.png
MagImg32.png
MagImg33.png
MagImg34.png
MagImg35.png
